View é uma consulta que pode ser usada como uma tabela, mas não armazena nenhum dado. Toda vez que uma view é usada, o SQLite executa a consulta que a define. Uma view pode ser usada para simplificar uma consulta complexa, ou para fornecer uma visão de dados que não está armazenada em uma tabela.

Uma view é criada usando a cláusula CREATE VIEW. A sintaxe é:

`CREATE [TEMP | TEMPORARY] VIEW nome_da_view AS`
    
    SELECT ...
    
    FROM ...
    
    WHERE [Condição]

Sendo apenas umas visualização, não é possível usar ISERT, UPDATE ou DELETE em uma view. 

In [1]:
import pandas as pd
import sqlite3 

In [2]:
con = sqlite3.connect('../primeiro_banco')
cur = con.cursor()
print('Conexão realizada com sucesso!')

Conexão realizada com sucesso!


In [5]:
query = 'SELECT name from sqlite_master where type = "table"'

df = pd.read_sql_query(query, con)
df

,name
0,Teste
1,Customers


In [7]:
query = 'SELECT * FROM Customers LIMIT 3'

df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,90000.0,2,Bachelors,Professional,Y
1,11001,EUGENE,HUANG,5/14/1965,S,M,eugene10@adventure-works.com,60000.0,3,Bachelors,Professional,N
2,11002,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,60000.0,3,Bachelors,Professional,Y


Simplificação das consultas


Consulta que será simplificada

In [ ]:
query = '''
SELECT FirstName, LastName, MaritalStatus, Gender, Occupation, AnnualIncome, IncomeLevel, COUNT(IncomeLevel) as TotalLevel
FROM(
SELECT FirstName, LastName, MaritalStatus, Gender, Occupation, AnnualIncome,
CASE 
    WHEN (AnnualIncome) > 260000 THEN 'A'
    WHEN (AnnualIncome) BETWEEN 84000 AND 260000 THEN 'B'
    WHEN (AnnualIncome) BETWEEN 36000 AND 84000 THEN 'C'
    WHEN (AnnualIncome) < 36000 THEN 'D'
    ELSE 'não informado'
END IncomeLevel
FROM Customers)
GROUP BY IncomeLevel
LIMIT 10
'''

df = pd.read_sql_query(query, con)
df

,FirstName,LastName,MaritalStatus,Gender,Occupation,AnnualIncome,IncomeLevel,TotalLevel
0,JON,YANG,M,M,Professional,90000.0,B,2973
1,EUGENE,HUANG,S,M,Professional,60000.0,C,10059
2,CHLOE,YOUNG,S,F,Skilled Manual,30000.0,D,5116


Criação da view

In [19]:
con.executescript('''
    CREATE VIEW GrupoRenda_VW AS SELECT FirstName, LastName, MaritalStatus, Gender, Occupation, AnnualIncome, IncomeLevel, COUNT(IncomeLevel) as TotalLevel
FROM(
SELECT FirstName, LastName, MaritalStatus, Gender, Occupation, AnnualIncome,
CASE 
    WHEN (AnnualIncome) > 260000 THEN 'A'
    WHEN (AnnualIncome) BETWEEN 84000 AND 260000 THEN 'B'
    WHEN (AnnualIncome) BETWEEN 36000 AND 84000 THEN 'C'
    WHEN (AnnualIncome) < 36000 THEN 'D'
    ELSE 'não informado'
END IncomeLevel
FROM Customers)
GROUP BY IncomeLevel
LIMIT 10
'''
)

con.commit()

In [21]:
query = 'SELECT * FROM GrupoRenda_VW'

df = pd.read_sql_query(query, con)
df

,FirstName,LastName,MaritalStatus,Gender,Occupation,AnnualIncome,IncomeLevel,TotalLevel
0,JON,YANG,M,M,Professional,90000.0,B,2973
1,EUGENE,HUANG,S,M,Professional,60000.0,C,10059
2,CHLOE,YOUNG,S,F,Skilled Manual,30000.0,D,5116


Cálculos com View

In [23]:
query = 'SELECT SUM(TotalLevel) AS Total FROM GrupoRenda_VW'

df = pd.read_sql_query(query, con)
df

,Total
0,18148


Deletar uma view

In [24]:
con.execute('DROP VIEW GrupoRenda_VW')

con.commit()